<a href="https://colab.research.google.com/github/TracyWu7724/stats507-coursework/blob/main/finetuned_t5_cnn_bo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [ ]:
!pip install transformers evaluate datasets transformers[torch] rouge_score nltk optuna

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=09d010536d53f4adf3c586dd1a2b3f240a61923d430e2c0345c23521f6c702ad
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a

## Load dataset

In [ ]:
from datasets import load_dataset

ds = load_dataset("abisee/cnn_dailymail", "3.0.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

### dataset shape

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

### dataset sample

In [ ]:
ds["train"][0]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

## Load Tokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

## Preprocess Data

In [ ]:
def tokenize_input(example):
    start_prompt = "Summarize the conversation. \n\n"
    end_prompt = "\n\nSummary: "
    prompt = [start_prompt + article + end_prompt for article in example["article"]]
    inputs = tokenizer(prompt, padding="max_length", truncation=True, return_tensors='pt', max_length=512)
    labels = tokenizer(example["highlights"], padding="max_length", truncation=True, return_tensors='pt', max_length=128)

    # Assign the tokenized input ids and labels
    example['input_ids'] = inputs.input_ids.squeeze()  # Remove extra dimension
    example['labels'] = labels.input_ids.squeeze()  # Remove extra dimension

    return example

tokenizer.pad_token = tokenizer.eos_token
tokenized_datasets = ds.map(tokenize_input, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['article', 'highlights', 'id'])

# Simplify the tokenized_dataset for lighter computing workloads
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10))

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
small_train_dataset[0].keys()

dict_keys(['input_ids', 'labels'])

## load Model

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Train and Evaluate

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate

import nltk
import numpy as np
nltk.download("punkt")
nltk.download('punkt_tab')

import optuna

metric = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


def objective(trial: optuna.Trial):
    # Hyperparameters to optimize
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])  # Example values for batch size
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)  # Log scale for learning rate
    num_train_epochs = trial.suggest_int('num_train_epochs', 3, 5)  # Epochs range

    args = Seq2SeqTrainingArguments(
        f"t5small-finetuned-cnn",
        eval_strategy = "epoch",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        predict_with_generate=True,
        fp16=True,
        push_to_hub=False,
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        compute_metrics=compute_metrics,
        train_dataset=small_train_dataset,
        eval_dataset=small_eval_dataset
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_result = trainer.evaluate()

    return eval_result["eval_rouge2"]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
study = optuna.create_study(direction="maximize")  # We want to maximize accuracy
study.optimize(objective, n_trials=10)  # Number of trials you want Optuna to run
print(f"Best hyperparameters: {study.best_params}")

[I 2024-12-02 00:31:36,149] A new study created in memory with name: no-name-24bb1f1a-34d8-44ac-95dd-1e81c27cced4
<ipython-input-10-3b0c8e4a6d41>:40: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)  # Log scale for learning rate
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more 

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

[I 2024-12-02 00:35:02,530] Trial 8 finished with value: 4.346 and parameters: {'batch_size': 32, 'learning_rate': 1.3617727343243628e-05, 'num_train_epochs': 4}. Best is trial 1 with value: 5.9542.
<ipython-input-10-3b0c8e4a6d41>:40: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)  # Log scale for learning rate
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.963409,19.140100,3.893200,15.573700,17.741300,20.000000
2,No log,0.960568,18.536200,3.965100,15.014600,17.129800,20.000000
3,No log,0.959456,18.536200,3.965100,15.014600,17.129800,20.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[I 2024-12-02 00:35:17,452] Trial 9 finished with value: 3.9651 and parameters: {'batch_size': 8, 'learning_rate': 5.7830857634487666e-05, 'num_train_epochs': 4}. Best is trial 1 with value: 5.9542.


Best hyperparameters: {'batch_size': 32, 'learning_rate': 1.3564453752292787e-05, 'num_train_epochs': 3}


### Train with optimized hyperparameters

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
best_batch_size = 32

args = Seq2SeqTrainingArguments(
    f"t5small-cnn-BO-finetuned",
    eval_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        compute_metrics=compute_metrics,
        train_dataset=small_train_dataset,
        eval_dataset=small_eval_dataset
    )

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.985473,19.656900,4.913900,16.066700,17.707900,20.000000
2,No log,0.986091,19.605700,4.427800,15.572200,17.660500,20.000000
3,No log,0.986129,19.605700,4.427800,15.572200,17.660500,20.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=12, training_loss=1.0194776852925618, metrics={'train_runtime': 14.9556, 'train_samples_per_second': 20.059, 'train_steps_per_second': 0.802, 'total_flos': 40602540441600.0, 'train_loss': 1.0194776852925618, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.9861286282539368,
 'eval_rouge1': 19.6057,
 'eval_rouge2': 4.4278,
 'eval_rougeL': 15.5722,
 'eval_rougeLsum': 17.6605,
 'eval_gen_len': 20.0,
 'eval_runtime': 0.9835,
 'eval_samples_per_second': 10.168,
 'eval_steps_per_second': 1.017,
 'epoch': 3.0}

## Load the trained model to Hugging Face

In [ ]:
model.push_to_hub("t5small-cnn-BO-finetuned")

CommitInfo(commit_url='https://huggingface.co/TracyWu32/t5small-cnn-BO-finetuned/commit/23403906c9c9446df924a3e51b83dbfbddc405d2', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='23403906c9c9446df924a3e51b83dbfbddc405d2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/TracyWu32/t5small-cnn-BO-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='TracyWu32/t5small-cnn-BO-finetuned'), pr_revision=None, pr_num=None)

## Inference real-world data

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# it can take 10mins to run the scrape function.
def scrape_comapny_desc(raw_file):
    company_profile = dict()
    headers =  {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36' }

    raw_sp = pd.read_excel(raw_file)

    for i in raw_sp.Symbol:
        link = "https://finance.yahoo.com/quote/" + str(i) + "/profile/"
        yahoo_url = requests.get(link, headers=headers)
        soup = BeautifulSoup(yahoo_url.text, 'html.parser')
        description_section = soup.find('section', {'data-testid': 'description'})

        if description_section:
            description_paragraph = description_section.find('p')
            company_profile[i] = description_paragraph.get_text(strip=True)
        else:
            continue

    raw_company_profile = pd.DataFrame.from_dict(company_profile, orient='index', columns=['Profile'])
    df_merged = raw_company_profile.merge(raw_sp, how='inner', left_index=True, right_on='Symbol')
    df_clean = df_merged[['Company', 'Symbol','Profile']]
    df_clean.to_csv("/Users/tracy/Desktop/UMich/STATS 507/Final_Proj/Real/data_Profile.csv", index=0)

    return df_clean

def get_summary(profile_file, model, tokenizer):
    df_desc = pd.read_csv(profile_file)

    summaries = []

    for p in df_desc.Profile:
        inputs = tokenizer(p, return_tensors="pt", max_length=500, truncation=True)
        summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=100, early_stopping=True)
        summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        summaries.append(summary)

    df_desc["Summary"] = summaries

    return df_desc

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

from google.colab import drive
drive.mount('/content/drive')

def main():
    # raw_file_path = "/Users/tracy/Desktop/UMich/STATS 507/Final_Proj/Real/data_SP500.xlsx"
    # df_desc = scrape_comapny_desc(raw_file_path)
    # desc_path = "/Users/tracy/Desktop/UMich/STATS 507/Final_Proj/Real/data_Profile.csv"
    desc_path = "/content/drive/My Drive/Colab Notebooks/data_Profile.csv"
    model = AutoModelForSeq2SeqLM.from_pretrained("TracyWu32/t5small-cnn-BO-finetuned")
    tokenizer = AutoTokenizer.from_pretrained("t5-small")
    df_final = get_summary(desc_path, model, tokenizer)
    df_final.to_csv("/content/drive/My Drive/Colab Notebooks/data_Summary.csv")

    return df_final

main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Company,Symbol,Profile,Summary
0,Nvidia Corp,NVDA,NVIDIA Corporation provides graphics and compu...,NVIDIA Corporation provides graphics and compu...
1,Apple Inc.,AAPL,"Apple Inc. designs, manufactures, and markets ...","Apple Inc. designs, manufactures, and markets ..."
2,Microsoft Corp,MSFT,Microsoft Corporation develops and supports so...,The Productivity and Business Processes segmen...
3,Amazon.com Inc,AMZN,"Amazon.com, Inc. engages in the retail sale of...","Amazon.com, Inc. engages in the retail sale of..."
4,"Meta Platforms, Inc. Class A",META,"Meta Platforms, Inc. engages in the developmen...","Meta Platforms, Inc. operates in two segments,..."
...,...,...,...,...
105,"Altria Group, Inc.",MO,"Altria Group, Inc., through its subsidiaries, ...","Altria Group, Inc., through its subsidiaries, ..."
106,"Nike, Inc.",NKE,"NIKE, Inc., together with its subsidiaries, en...","NIKE, Inc., together with its subsidiaries, en..."
107,Intercontinental Exchange Inc.,ICE,"Intercontinental Exchange, Inc., together with...","Intercontinental Exchange, Inc., together with..."
108,The Sherwin-Williams Company,SHW,The Sherwin-Williams Company engages in the de...,The Sherwin-Williams Company operates through ...
